In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import time
import pandas as pd
from helper import get_penmap

Using TensorFlow backend.


In [2]:
def compute_dist(param1,param2):
    d = 0
    for i in range(0,len(param1)):
        d+=np.nan_to_num(((param1[i]-param2[i])**2)/(param1[i]+param2[i]))
    return d

def dist(param1,param2,n1,n2):
    mean1 = np.mean(param1)
    mean2 = np.mean(param2)
    var1  = np.var(param1)
    var2 = np.var(param2)
     
    d = np.abs(mean1-mean2)
    d/=(var1/n1 + var2/n2)**0.5
    
    return d

def distance_distribution(features):
    same_pens = {}
    diff_pens = {}
    
    for k in range(1,113) :
        for i in ['1','2']: 
            for n in ['1','2']:
                for j in ['1','2','3','4','5','6','7','8'] :
                    for m in ['1','2','3','4','5','6','7','8'] :
                        try:
                            if(i==n and j==m):
                                continue    
                            elif i==n and 'c_'+str(k)+'_'+n+'_'+m+'::'+i+'_'+j not in same_pens:
                                same_pens['c_'+str(k)+'_'+i+'_'+j+'::'+n+'_'+m] = np.nan_to_num(compute_dist(features['c'+str(k)+'_'+i+'_'+j],features['c'+str(k)+'_'+n+'_'+m]))
                    
                            elif i!=n and 'c_'+str(k)+'_'+n+'_'+m+'::'+i+'_'+j not in diff_pens :                               
                                diff_pens['c_'+str(k)+'_'+i+'_'+j+'::'+n+'_'+m] = np.nan_to_num(compute_dist(features['c'+str(k)+'_'+i+'_'+j],features['c'+str(k)+'_'+n+'_'+m]))
                        except:
                            
                            pass
     
    
    same_pens = [ v for v in same_pens.values() ]
    diff_pens = [ v for v in diff_pens.values() ]
   # print('Lengths same pens-{} different pens-{}'.format(len(same_pens),len(diff_pens)))
    return same_pens,diff_pens

def pen_distribution(features,a,pen_out):
    same_pens = {}
    diff_pens = {}
    
    for k in range(1,113) :
        if(k in a[pen_out]):
            for i in ['1','2']: 
                for n in ['1','2']:
                    for j in ['1','2','3','4','5','6','7','8'] :
                        for m in ['1','2','3','4','5','6','7','8'] :
                            try:
                                if(i==n and j==m):
                                    continue    
                                elif i==n and 'c_'+str(k)+'_'+n+'_'+m+'::'+i+'_'+j not in same_pens:
                                    same_pens['c_'+str(k)+'_'+i+'_'+j+'::'+n+'_'+m] = compute_dist(features['c'+str(k)+'_'+i+'_'+j],features['c'+str(k)+'_'+n+'_'+m])

                                elif i!=n and 'c_'+str(k)+'_'+n+'_'+m+'::'+i+'_'+j not in diff_pens :                               
                                    diff_pens['c_'+str(k)+'_'+i+'_'+j+'::'+n+'_'+m] = compute_dist(features['c'+str(k)+'_'+i+'_'+j],features['c'+str(k)+'_'+n+'_'+m])
                            except:

                                pass

    
    same_pens = [ v for v in same_pens.values() ]
    diff_pens = [ v for v in diff_pens.values() ]
   # print('Lengths same pens-{} different pens-{}'.format(len(same_pens),len(diff_pens)))
    return same_pens,diff_pens

In [3]:
def distance_btw_distributions(feature,a):
    d = []
    same_pens,diff_pens = distance_distribution(feature)
    D = dist(same_pens,diff_pens,1465,1456)
    for j in range(1,15):
        same_pens_i,diff_pens_i= pen_distribution(feature,a,j)
        d1 = len(same_pens_i)/2930*dist(same_pens_i,same_pens,len(same_pens_i),1465)
        d2 = len(diff_pens_i)/2912*dist(diff_pens_i,diff_pens,len(diff_pens_i),1456)
        d.append(d1+d2)
        
    return D-sum(d)

In [4]:
def load_parameters(path,n_param):
    hist_Y = {}
    hist_Cb = {}
    hist_Cr = {}
    a = pd.read_csv(path)
    for k in range(1,113):
        for i in ['1','2']:
            for j in ['1','2','3','4','5','6','7','8']:
                try:
                    
                    hist_Y['c'+str(k)+'_'+i+'_'+j] = a['c'+str(k)+'_'+i+'_'+j][0:n_param] 
                    hist_Cb['c'+str(k)+'_'+i+'_'+j] = a['c'+str(k)+'_'+i+'_'+j][n_param:2*n_param]
                    hist_Cr['c'+str(k)+'_'+i+'_'+j] = a['c'+str(k)+'_'+i+'_'+j][2*n_param:3*n_param]
                    
                    hist_Y['c'+str(k)+'_'+i+'_'+j].reset_index(inplace=True, drop=True)
                    hist_Cb['c'+str(k)+'_'+i+'_'+j].reset_index(inplace=True, drop=True)
                    hist_Cr['c'+str(k)+'_'+i+'_'+j].reset_index(inplace=True, drop=True)
                    
                except:
                    break
    return hist_Y,hist_Cb,hist_Cr

In [5]:
def separate(h1,f):
    hist = {}    
    for k in range(1,113):
        for i in ['1','2']:
            for j in ['1','2','3','4','5','6','7','8']:
                try:
                    hist['c'+str(k)+'_'+i+'_'+j] = np.array([h1['c'+str(k)+'_'+i+'_'+j][8*f:(f+1)*8]]).reshape(8,)
                except:
                    break
    return hist

def concat2(h1,h2):
    hist = {}    
    for k in range(1,113):
        for i in ['1','2']:
            for j in ['1','2','3','4','5','6','7','8']:
                try:
                    hist['c'+str(k)+'_'+i+'_'+j]= np.concatenate([h1['c'+str(k)+'_'+i+'_'+j],h2['c'+str(k)+'_'+i+'_'+j]])
                
                except:
                    break
    return hist

def concat3(h1,h2,h3):
    hist = {}    
    for k in range(1,113):
        for i in ['1','2']:
            for j in ['1','2','3','4','5','6','7','8']:
                try:
                    hist['c'+str(k)+'_'+i+'_'+j] = np.concatenate([h1['c'+str(k)+'_'+i+'_'+j],h2['c'+str(k)+'_'+i+'_'+j],h3['c'+str(k)+'_'+i+'_'+j]])
                except:
                    break
    return hist


In [6]:
a = get_penmap()

In [7]:
path = '/home/mohit/Desktop/featuresUpdated/Gabor.csv'
y,cb,cr = load_parameters(path,200)

In [11]:
fe = {}

for i in range(1,26):
    fe[i]    = separate(y,i-1)
    fe[i+25] = separate(cb,i-1)
    fe[i+50] = separate(cr,i-1)
    

In [14]:
# min max normalising every feature
for i in range(1,76):
    for j in range(0,8):
        mi = np.min([ v[j] for v in fe[i].values() ])
        mx = np.max([ v[j] for v in fe[i].values() ])
        for key in fe[i].keys():
            fe[i][key][j]-=mi
            fe[i][key][j]/=(mx-mi)   

In [16]:
Distances = {}

for i in range(1,76):
    Distances[i] = distance_btw_distributions(fe[i],a)
        
sorted_by_value = sorted(Distances.items(), key=lambda kv: kv[1],reverse =True)
sorted_by_value[0]

(46, 22.323878230290475)

### after comparing all features, we found that fe[46] i.e 21_Cb is the best and will be the starting point of our forward selection technique.

In [17]:
fecb = {}
feycb = {}
fecbcr  ={}
feycbcr = {}
for i in range(1,26):
    fecb[i] = fe[i+25]
    feycb[i] = concat2(fe[i],fe[i+25])
    fecbcr[i] = concat2(fe[i+25],fe[i+50])
    feycbcr[i] = concat3(fe[i],fe[i+25],fe[i+50])

In [18]:
# fmax will contain the selected features. dmax will contain the increment by adding the ith feature. 
# Fmax will contain a dict with all selected features concatenated together.

fmax = [21]
dmax = [22.323878230290468]
Fmax = fecb[21]
Fmaxycb = feycb[21]
Fmaxcbcr = fecbcr[21]
Fmaxycbcr = feycbcr[21]

In [19]:
Dmax = sum(dmax)
f = range(1,26)

In [20]:
while(1):
    Distancescb = {}
    
    for i in set(f)-set(fmax):
        Distancescb[i] = distance_btw_distributions(concat2(Fmax,fecb[i]),a) - Dmax
        
    sorted_by_value = sorted(Distancescb.items(), key=lambda kv: kv[1],reverse =True)
    index = sorted_by_value[0][0]
    dd = sorted_by_value[0][1]
    
    Distancesycb = distance_btw_distributions(Fmaxycb,a)-Dmax
        
    Distancescbcr = distance_btw_distributions(Fmaxcbcr,a)-Dmax
    
    print(index,dd,Distancesycb,Distancescbcr)
    
    if(dd<Distancesycb or dd<Distancescbcr or dd<0):
        if(max(Distancesycb,Distancescbcr)>0):
            dmax.append(max(Distancesycb,Distancescbcr))
            Dmax = sum(dmax)
        break            
                
    fmax.append(index)
    dmax.append(dd)
    Fmax = concat2(fecb[index],Fmax)
    Dmax = sum(dmax)
    Fmaxycb = concat2(feycb[index],Fmaxycb)
    Fmaxcbcr = concat2(fecbcr[index],Fmaxcbcr)
    Fmaxycbcr = concat2(feycbcr[index],Fmaxycbcr)

24 1.0884680316693114 -1.6572025065105684 -6.31435665205106
22 0.5044139930732818 -2.345886294161552 -3.7131894665309098
16 -0.15804421915316524 -2.663681252184425 -4.656133416765698


In [21]:
fmax,dmax,Dmax

([21, 24, 22],
 [22.323878230290468, 1.0884680316693114, 0.5044139930732818],
 23.91676025503306)

### after applying our strategy, we found the following selection pattern. 

### 21cb -> 24 cb -> 22cb

### so our subset selected will be [21,22,24] in cb channel only

In [23]:
# So lets save the original features(not normalised)!

best = concat3(separate(cb,20),separate(cb,21),separate(cb,23))
best

{'c100_1_1': array([-342.49456496,   12.95009874,  -42.98362854,   10.16787256,
        -341.95082936,   13.35645872,  -42.43087608,    8.45933483,
         348.79363281,   11.51053628,  -76.24067127,   12.30550015,
         348.49033045,   10.43134345,  -75.15990082,    9.97880936,
        -435.73042895,   13.10282474, -124.79435239,   14.1553002 ,
        -435.06960723,   11.92989867, -122.57824179,   11.85651913]),
 'c100_1_2': array([-342.59390162,   13.49696495,  -42.82571941,    8.79339667,
        -342.70823926,   12.95663876,  -42.47128453,    8.19128816,
         349.12350141,   11.90445204,  -75.86782338,   11.02131922,
         349.37629611,   10.73622096,  -75.45298455,   10.09999175,
        -436.13173604,   14.87656106, -123.56941353,   12.1922186 ,
        -436.09907731,   11.41968461, -123.34451681,   14.85517131]),
 'c100_1_3': array([-342.89448544,   17.01457823,  -42.50166061,    8.20852003,
        -343.15170718,   19.05965938,  -42.15816364,    7.37985343,
        

In [24]:
best = pd.DataFrame(best)
best

,c100_1_1,c100_1_2,c100_1_3,c100_1_4,c100_2_1,c100_2_2,c100_2_3,c101_1_1,c101_1_2,c101_1_3,...,c99_2_3,c9_1_1,c9_1_2,c9_1_3,c9_1_4,c9_1_5,c9_1_6,c9_1_7,c9_2_1,c9_2_2
0,-342.494565,-342.593902,-342.894485,-342.320487,-336.342974,-336.128916,-336.194087,-337.365767,-337.628423,-337.421771,...,-336.754226,-329.972673,-328.610949,-328.203725,-326.131637,-327.349725,-327.594023,-326.645907,-323.361461,-322.253283
1,12.950099,13.496965,17.014578,17.412060,14.375173,21.015705,19.477954,13.795094,13.407772,11.544499,...,17.216644,18.018839,15.955056,20.623771,25.327128,22.581697,25.143163,23.032068,16.072102,30.883904
2,-42.983629,-42.825719,-42.501661,-42.528222,-41.684489,-41.636987,-41.609322,-41.861530,-41.888015,-41.777426,...,-41.682382,-40.128432,-40.417090,-40.002332,-40.048012,-39.835986,-40.392720,-39.974700,-40.196803,-39.471214
3,10.167873,8.793397,8.208520,8.839559,6.977465,8.042703,8.054650,7.113555,7.205827,6.424438,...,7.625227,10.303749,9.715966,11.465770,10.197284,11.784880,10.067600,10.567237,9.504589,11.309643
4,-341.950829,-342.708239,-343.151707,-343.220352,-336.136641,-335.956943,-335.968638,-337.406195,-337.503345,-337.136356,...,-336.135608,-329.574250,-326.926885,-327.416101,-325.616878,-327.022642,-327.875305,-326.823762,-326.079273,-318.015234
5,13.356459,12.956639,19.059659,15.369588,11.042377,11.371244,11.325965,9.911634,10.619470,11.159914,...,8.851653,12.467758,17.153308,16.774786,16.427695,16.717150,15.064099,17.750356,20.997004,18.178354
6,-42.430876,-42.471285,-42.158164,-42.425474,-41.727130,-41.603869,-41.533454,-41.705501,-41.717620,-41.732189,...,-41.645431,-39.767247,-39.404200,-39.900741,-39.733947,-40.071212,-40.550724,-40.043166,-40.025741,-39.888082
7,8.459335,8.191288,7.379853,8.510208,8.965630,9.258281,9.024494,6.800077,8.213121,7.632579,...,7.438522,9.856557,11.937798,11.083284,10.767388,10.944295,9.657756,11.881483,10.727791,11.886975
8,348.793633,349.123501,348.493583,348.420282,343.482767,343.490954,343.365086,344.349900,344.865107,344.810592,...,343.016944,340.230094,337.972009,339.164466,337.138940,338.032579,338.308144,336.985675,332.296127,336.626685
9,11.510536,11.904452,11.927763,14.684668,13.456999,16.482492,16.886631,9.787859,10.799942,9.674404,...,13.106610,13.305523,12.135283,16.671989,20.380181,18.680485,19.769509,18.131254,14.617860,25.007927


In [25]:
best.to_csv('/home/mohit/Desktop/featuresUpdated/Gabor_Selectedfeatures.csv')